In [96]:
import ast
import pandas as pd
from datasets import load_dataset

In [97]:
# Load the 'recetasdelaabuela' dataset from Hugging Face
dataset = load_dataset("somosnlp/recetasdelaabuela", 'version_inicial')
# Convert the dataset to a pandas DataFrame
df = pd.DataFrame(dataset['train'])

# Display the dataset structure
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Id', 'Nombre', 'URL', 'Ingredientes', 'Pasos', 'Pais', 'Duracion', 'Porciones', 'Calorias', 'Categoria', 'Contexto', 'Valoracion y Votos', 'Comensales', 'Tiempo', 'Dificultad', 'Categoria 2', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18'],
        num_rows: 20085
    })
})


### Data Cleaning

In [98]:
# convert all column names to lowercase
df.columns = df.columns.str.lower()

# Drop 'Id' column and rename 'index' to Id
df.drop(columns=['id'], inplace=True)
df.reset_index(drop= True, inplace=True)
# df.index.name = 'id'


In [101]:
# name for a function to convert any column from a string representation of a list to a space-separated string.
def convert_list_to_string(input_string):
    """
    Convert a column from a string representation of a list to a space-separated string.
    """
    if isinstance(input_string, str) and '[' in input_string:
        # Convert the string representation of a list to an actual list
        try:
            return ' '.join(ast.literal_eval(input_string))
        except (ValueError, SyntaxError):
            # If conversion fails, pass and return the original value
            pass
    return input_string                                      

In [102]:
df['pasos'] = df['pasos'].apply(lambda ss: convert_list_to_string(ss)) # Convert the 'Pasos' column
df['ingredientes'] = df['ingredientes'].apply(lambda ss: convert_list_to_string(ss)) # Convert the 'Ingredientes' column


In [103]:
df[df['pais']=='Mexico'].head()

,nombre,url,ingredientes,pasos,pais,duracion,porciones,calorias,categoria,contexto,valoracion y votos,comensales,tiempo,dificultad,categoria 2,unnamed: 16,unnamed: 17,unnamed: 18
983,tacos de huevo,https://www.recetasgratis.net/receta-de-tacos-...,4 huevos 1 pieza de tomate grande 1 chile serr...,1 Pica las verduras en cubos o trozos pequeños...,Mexico,15m,"['4 huevos', '1 pieza de tomate grande', '1 ch...",None,[],"Cuando se prepara el desayuno, el huevo es uno...",None,2 comensales,Desayuno,Dificultad muy baja,"Bajo en calorías, Sin grasa, Alto en proteínas",None,None,tacos de huevo
984,tacos al pastor con pina,https://www.recetasgratis.net/receta-de-tacos-...,500 gramos de carne de res o cerdo 4 piezas de...,1 Para hacer esta receta de tacos la pastor co...,Mexico,4h,"['500 gramos de carne de res o cerdo', '4 piez...",None,['Recetas mexicanas'],Los tacos al pastor son uno de los tacos más p...,None,5 comensales,Merienda,Dificultad baja,"Bajo en calorías, Sin grasa, Alto en fibra",None,None,tacos al pastor con pina
997,gorditas de maseca,https://www.recetasgratis.net/receta-de-gordit...,2 tazas de harina de maíz Maseca ½ taza de har...,1 Para preparar la masa: en un tazón mezcla la...,Mexico,2h 30m,"['2 tazas de harina de maíz Maseca', '½ taza d...",None,"['Receta sin azúcar', 'Receta sin gluten', 'Re...",Las gorditas de maseca son un delicioso platil...,None,3 comensales,Plato principal,Dificultad media,"Alto en grasas, Alto en sodio, Sin azúcar añadida",None,None,gorditas de maseca
998,quesadilla de chicharron,https://www.recetasgratis.net/receta-de-quesad...,250 gramos de chicharrón prensado molido 3 jit...,1 Suaviza los chiles remojándolos en agua cali...,Mexico,1h 30m,"['250 gramos de chicharrón prensado molido', '...",None,"['Receta sin azúcar', 'Receta sin gluten', 'Re...",Las quesadillas son un platillo típico de la c...,None,5 comensales,Entrante,Dificultad media,"Bajo en calorías, Sin grasa, Alto en fibra",None,None,quesadilla de chicharron
999,tacos de pastor negro,https://www.recetasgratis.net/receta-de-tacos-...,6 chiles anchos ½ cucharadita de sal fina 3 pi...,"1 Para comenzar, retira las semillas de los ch...",Mexico,24h,"['6 chiles anchos', '½ cucharadita de sal fina...",None,"['Receta sin lactosa', 'Receta sin azúcar', 'R...",Los tacos de pastor negro son una variante de ...,None,4 comensales,Plato principal,Dificultad baja,"Alto en sodio, Alto en grasas saturadas, Sin a...",None,None,tacos de pastor negro


#### Further Data Cleaning Steps TODO

* Convert the `duracion` column to minutes from various formats (e.g., "45 min", "1h 30m").
* Handle multiple values in the `categoria` and `categoria 2` columns by splitting them into separate rows or creating lists.